In [1]:
import copy
import findspark
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, DateType
from pyspark.sql.functions import col, lit, explode, split , array, array_contains, \
    udf, map_values, when, count, min, max, avg, row_number, rank, dense_rank, percent_rank, \
    lead, lag, first, last, nth_value, from_json, map_keys, to_json, json_tuple
import pyspark.sql.functions as sql_functions

findspark.init()

In [2]:
spark = SparkSession.builder.appName('Spark SQL').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 20:25:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
left_df = spark.createDataFrame(
    data=[(1,'ali'),(2, 'akram'),(3, 'oveys'),(4, 'ala'),(5, 'omid'),(6, 'mobin')], schema=['id', 'name']
)

right_df = spark.createDataFrame(
        data=[(8,'mehdi'),(9, 'simin'),(3, 'oveys'),(4, 'ala'),(10, 'aida')], schema=['id', 'name']
)

In [4]:
# options are inner, outer, left, right
left_df.join(right_df, on=['id'], how='left').show()

+---+-----+-----+
| id| name| name|
+---+-----+-----+
|  1|  ali| NULL|
|  2|akram| NULL|
|  3|oveys|oveys|
|  4|  ala|  ala|
|  5| omid| NULL|
|  6|mobin| NULL|
+---+-----+-----+



In [5]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

rdd1 = spark.sparkContext.parallelize([(1,'a'),(2, 'b')])
rdd2 = spark.sparkContext.parallelize([(1,'d'),(3, 'b')])
rdd_res = rdd1.union(rdd2)

spark.createDataFrame(rdd_res, schema).show()



+---+----+
| id|name|
+---+----+
|  1|   a|
|  2|   b|
|  1|   d|
|  3|   b|
+---+----+



In [6]:
titanic_shcema = StructType()\
    .add('pid', 'integer') \
    .add('survived', 'integer') \
    .add('class', 'integer') \
    .add('name', 'string') \
    .add('sex', 'string') \
    .add('age', 'integer') \
    .add('sib', 'integer') \
    .add('parch', 'integer') \
    .add('tikcet', 'string') \
    .add('fare', 'float') \
    .add('cabin', 'string') \
    .add('embarked', 'string') \



titanic_df = spark.read.option('header', 'true').schema(titanic_shcema).csv('titanic.csv')
titanic_df.show()

+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|pid|survived|class|                name|   sex| age|sib|parch|          tikcet|   fare|cabin|embarked|
+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|  1|       0|    3|Braund, Mr. Owen ...|  male|  22|  1|    0|       A/5 21171|   7.25| NULL|       S|
|  2|       1|    1|Cumings, Mrs. Joh...|female|  38|  1|    0|        PC 17599|71.2833|  C85|       C|
|  3|       1|    3|Heikkinen, Miss. ...|female|  26|  0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|  4|       1|    1|Futrelle, Mrs. Ja...|female|  35|  1|    0|          113803|   53.1| C123|       S|
|  5|       0|    3|Allen, Mr. Willia...|  male|  35|  0|    0|          373450|   8.05| NULL|       S|
|  6|       0|    3|    Moran, Mr. James|  male|NULL|  0|    0|          330877| 8.4583| NULL|       Q|
|  7|       0|    1|McCarthy, Mr. Tim...|  male|  54|  0|    0| 

24/04/04 16:23:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


In [7]:
titanic_1 = titanic_df.alias('titanic_1')

titanic_1.show()

+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|pid|survived|class|                name|   sex| age|sib|parch|          tikcet|   fare|cabin|embarked|
+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|  1|       0|    3|Braund, Mr. Owen ...|  male|  22|  1|    0|       A/5 21171|   7.25| NULL|       S|
|  2|       1|    1|Cumings, Mrs. Joh...|female|  38|  1|    0|        PC 17599|71.2833|  C85|       C|
|  3|       1|    3|Heikkinen, Miss. ...|female|  26|  0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|  4|       1|    1|Futrelle, Mrs. Ja...|female|  35|  1|    0|          113803|   53.1| C123|       S|
|  5|       0|    3|Allen, Mr. Willia...|  male|  35|  0|    0|          373450|   8.05| NULL|       S|
|  6|       0|    3|    Moran, Mr. James|  male|NULL|  0|    0|          330877| 8.4583| NULL|       Q|
|  7|       0|    1|McCarthy, Mr. Tim...|  male|  54|  0|    0| 

24/04/04 16:23:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


In [8]:
titanic_1.select('cabin').show()

+-----+
|cabin|
+-----+
| NULL|
|  C85|
| NULL|
| C123|
| NULL|
| NULL|
|  E46|
| NULL|
| NULL|
| NULL|
|   G6|
| C103|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
+-----+
only showing top 20 rows



In [9]:
titanic_1.filter(titanic_1.age > 35).count()

210

In [10]:
titanic_1.select('name', 'age', 'survived').filter('survived = 0').show()

+--------------------+----+--------+
|                name| age|survived|
+--------------------+----+--------+
|Braund, Mr. Owen ...|  22|       0|
|Allen, Mr. Willia...|  35|       0|
|    Moran, Mr. James|NULL|       0|
|McCarthy, Mr. Tim...|  54|       0|
|Palsson, Master. ...|   2|       0|
|Saundercock, Mr. ...|  20|       0|
|Andersson, Mr. An...|  39|       0|
|Vestrom, Miss. Hu...|  14|       0|
|Rice, Master. Eugene|   2|       0|
|Vander Planke, Mr...|  31|       0|
|Fynney, Mr. Joseph J|  35|       0|
|Palsson, Miss. To...|   8|       0|
|Emir, Mr. Farred ...|NULL|       0|
|Fortune, Mr. Char...|  19|       0|
| Todoroff, Mr. Lalio|NULL|       0|
|Uruchurtu, Don. M...|  40|       0|
|Wheadon, Mr. Edwa...|  66|       0|
|Meyer, Mr. Edgar ...|  28|       0|
|Holverson, Mr. Al...|  42|       0|
|Cann, Mr. Ernest ...|  21|       0|
+--------------------+----+--------+
only showing top 20 rows



In [11]:
titanic_1.select('name', 'survived').where('age < 30 and survived=1').show()

+--------------------+--------+
|                name|survived|
+--------------------+--------+
|Heikkinen, Miss. ...|       1|
|Johnson, Mrs. Osc...|       1|
|Nasser, Mrs. Nich...|       1|
|Sandstrom, Miss. ...|       1|
|"McGowan, Miss. A...|       1|
|Sloper, Mr. Willi...|       1|
|Nicola-Yarred, Mi...|       1|
|Laroche, Miss. Si...|       1|
|Devaney, Miss. Ma...|       1|
|Faunthorpe, Mrs. ...|       1|
|   Rugg, Miss. Emily|       1|
|West, Miss. Const...|       1|
|Nye, Mrs. (Elizab...|       1|
|Andersson, Miss. ...|       1|
|Sheerlinck, Mr. J...|       1|
| Ilett, Miss. Bertha|       1|
|Fortune, Miss. Ma...|       1|
|Greenfield, Mr. W...|       1|
|Salkjelsvik, Miss...|       1|
|Nicola-Yarred, Ma...|       1|
+--------------------+--------+
only showing top 20 rows



In [12]:
titanic_1.sort(['age', 'name'], ascending=[False, True]).show()

+---+--------+-----+--------------------+------+---+---+-----+-----------+-------+-----------+--------+
|pid|survived|class|                name|   sex|age|sib|parch|     tikcet|   fare|      cabin|embarked|
+---+--------+-----+--------------------+------+---+---+-----+-----------+-------+-----------+--------+
|631|       1|    1|Barkworth, Mr. Al...|  male| 80|  0|    0|      27042|   30.0|        A23|       S|
|852|       0|    3| Svensson, Mr. Johan|  male| 74|  0|    0|     347060|  7.775|       NULL|       S|
|494|       0|    1|Artagaveytia, Mr....|  male| 71|  0|    0|   PC 17609|49.5042|       NULL|       C|
| 97|       0|    1|Goldschmidt, Mr. ...|  male| 71|  0|    0|   PC 17754|34.6542|         A5|       C|
|746|       0|    1|Crosby, Capt. Edw...|  male| 70|  1|    1|  WE/P 5735|   71.0|        B22|       S|
|673|       0|    2|Mitchell, Mr. Hen...|  male| 70|  0|    0| C.A. 24580|   10.5|       NULL|       S|
| 34|       0|    2|Wheadon, Mr. Edwa...|  male| 66|  0|    0| C

24/04/04 16:23:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


In [13]:
titanic_1.describe().show()

24/04/04 16:23:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/04/04 16:23:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|              pid|           survived|             class|                name|   sex|               age|               sib|              parch|            tikcet|             fare|cabin|embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               689|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                NULL|  NULL|29.847605224963715|0.5230078563411896|0.38159371492704824|260318.54916792738|32.204208

In [14]:
titanic_1.columns

['pid',
 'survived',
 'class',
 'name',
 'sex',
 'age',
 'sib',
 'parch',
 'tikcet',
 'fare',
 'cabin',
 'embarked']

In [15]:
titanic_1.printSchema()

root
 |-- pid: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- class: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sib: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- tikcet: string (nullable = true)
 |-- fare: float (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)



In [16]:
help(titanic_1.withColumn)

Help on method withColumn in module pyspark.sql.dataframe:

withColumn(colName: str, col: pyspark.sql.column.Column) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` by adding a column or replacing the
    existing column that has the same name.
    
    The column expression must be an expression over this :class:`DataFrame`; attempting to add
    a column from some other :class:`DataFrame` will raise an error.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    colName : str
        string, name of the new column.
    col : :class:`Column`
        a :class:`Column` expression for the new column.
    
    Returns
    -------
    :class:`DataFrame`
        DataFrame with new or replaced column.
    
    Notes
    -----
    This method introduces a projection internally. Therefore, calling it multiple
    times, for instance, via loops in order to 

# withColumn()

##### It's a transformation operation which is used for 
##### 1. adding a new column
##### 2. changing the dtype of an existing column
##### 3. changing the value of an existing column

In [17]:
# change dtype
titanic_1.withColumn(colName='sib', col=titanic_1.sib.cast('float')).show()

+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|pid|survived|class|                name|   sex| age|sib|parch|          tikcet|   fare|cabin|embarked|
+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|  1|       0|    3|Braund, Mr. Owen ...|  male|  22|1.0|    0|       A/5 21171|   7.25| NULL|       S|
|  2|       1|    1|Cumings, Mrs. Joh...|female|  38|1.0|    0|        PC 17599|71.2833|  C85|       C|
|  3|       1|    3|Heikkinen, Miss. ...|female|  26|0.0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|  4|       1|    1|Futrelle, Mrs. Ja...|female|  35|1.0|    0|          113803|   53.1| C123|       S|
|  5|       0|    3|Allen, Mr. Willia...|  male|  35|0.0|    0|          373450|   8.05| NULL|       S|
|  6|       0|    3|    Moran, Mr. James|  male|NULL|0.0|    0|          330877| 8.4583| NULL|       Q|
|  7|       0|    1|McCarthy, Mr. Tim...|  male|  54|0.0|    0| 

24/04/04 16:23:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


In [18]:
# data manipulation
titanic_1.withColumn('sib', titanic_1.sib * 1).show()

+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|pid|survived|class|                name|   sex| age|sib|parch|          tikcet|   fare|cabin|embarked|
+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|  1|       0|    3|Braund, Mr. Owen ...|  male|  22|  1|    0|       A/5 21171|   7.25| NULL|       S|
|  2|       1|    1|Cumings, Mrs. Joh...|female|  38|  1|    0|        PC 17599|71.2833|  C85|       C|
|  3|       1|    3|Heikkinen, Miss. ...|female|  26|  0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|  4|       1|    1|Futrelle, Mrs. Ja...|female|  35|  1|    0|          113803|   53.1| C123|       S|
|  5|       0|    3|Allen, Mr. Willia...|  male|  35|  0|    0|          373450|   8.05| NULL|       S|
|  6|       0|    3|    Moran, Mr. James|  male|NULL|  0|    0|          330877| 8.4583| NULL|       Q|
|  7|       0|    1|McCarthy, Mr. Tim...|  male|  54|  0|    0| 

24/04/04 16:23:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


In [19]:
# creating new col
titanic_1.withColumn(colName='nationality', col=lit('XYZ')).show()

+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+-----------+
|pid|survived|class|                name|   sex| age|sib|parch|          tikcet|   fare|cabin|embarked|nationality|
+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+-----------+
|  1|       0|    3|Braund, Mr. Owen ...|  male|  22|  1|    0|       A/5 21171|   7.25| NULL|       S|        XYZ|
|  2|       1|    1|Cumings, Mrs. Joh...|female|  38|  1|    0|        PC 17599|71.2833|  C85|       C|        XYZ|
|  3|       1|    3|Heikkinen, Miss. ...|female|  26|  0|    0|STON/O2. 3101282|  7.925| NULL|       S|        XYZ|
|  4|       1|    1|Futrelle, Mrs. Ja...|female|  35|  1|    0|          113803|   53.1| C123|       S|        XYZ|
|  5|       0|    3|Allen, Mr. Willia...|  male|  35|  0|    0|          373450|   8.05| NULL|       S|        XYZ|
|  6|       0|    3|    Moran, Mr. James|  male|NULL|  0|    0|         

24/04/04 16:23:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


# withColumnRename()

In [20]:
titanic_1.withColumnRenamed(existing='tikcet', new='pnr').show()

+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|pid|survived|class|                name|   sex| age|sib|parch|             pnr|   fare|cabin|embarked|
+---+--------+-----+--------------------+------+----+---+-----+----------------+-------+-----+--------+
|  1|       0|    3|Braund, Mr. Owen ...|  male|  22|  1|    0|       A/5 21171|   7.25| NULL|       S|
|  2|       1|    1|Cumings, Mrs. Joh...|female|  38|  1|    0|        PC 17599|71.2833|  C85|       C|
|  3|       1|    3|Heikkinen, Miss. ...|female|  26|  0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|  4|       1|    1|Futrelle, Mrs. Ja...|female|  35|  1|    0|          113803|   53.1| C123|       S|
|  5|       0|    3|Allen, Mr. Willia...|  male|  35|  0|    0|          373450|   8.05| NULL|       S|
|  6|       0|    3|    Moran, Mr. James|  male|NULL|  0|    0|          330877| 8.4583| NULL|       Q|
|  7|       0|    1|McCarthy, Mr. Tim...|  male|  54|  0|    0| 

24/04/04 16:23:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
 Schema: pid, survived, class, name, sex, age, sib, parch, tikcet, fare, cabin, embarked
Expected: pid but found: PassengerId
CSV file: file:///Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/src/pyspark_df/titanic.csv


# explode()

In [21]:
data = [(1, 'Oveys', ['java', 'python']), (2, 'Mohammad', ['Java', 'Python', 'Docker']), (3, 'Ala',['ObjectiveC', 'Swift', 'swiftUI', 'iOS'])]
devs = spark.createDataFrame(data, ['id', 'name', 'skills'])
devs.show()
devs.printSchema()

+---+--------+--------------------+
| id|    name|              skills|
+---+--------+--------------------+
|  1|   Oveys|      [java, python]|
|  2|Mohammad|[Java, Python, Do...|
|  3|     Ala|[ObjectiveC, Swif...|
+---+--------+--------------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [22]:
devs.withColumn('skills', explode(devs.skills)).show()

+---+--------+----------+
| id|    name|    skills|
+---+--------+----------+
|  1|   Oveys|      java|
|  1|   Oveys|    python|
|  2|Mohammad|      Java|
|  2|Mohammad|    Python|
|  2|Mohammad|    Docker|
|  3|     Ala|ObjectiveC|
|  3|     Ala|     Swift|
|  3|     Ala|   swiftUI|
|  3|     Ala|       iOS|
+---+--------+----------+



# split()

In [23]:
data = [(1, 'Oveys', 'java,python'), (2, 'Mohammad', 'Java,Python,Docker'), (3, 'Ala','ObjectiveC,Swift,swiftUI,iOS')]
devs = spark.createDataFrame(data, ['id', 'name', 'skills'])
devs.show()
devs.printSchema()

+---+--------+--------------------+
| id|    name|              skills|
+---+--------+--------------------+
|  1|   Oveys|         java,python|
|  2|Mohammad|  Java,Python,Docker|
|  3|     Ala|ObjectiveC,Swift,...|
+---+--------+--------------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: string (nullable = true)



In [24]:
devs.withColumn('skills', split('skills', ',')).show()

+---+--------+--------------------+
| id|    name|              skills|
+---+--------+--------------------+
|  1|   Oveys|      [java, python]|
|  2|Mohammad|[Java, Python, Do...|
|  3|     Ala|[ObjectiveC, Swif...|
+---+--------+--------------------+



# array()

In [25]:
data = [(1, 'Oveys', 'java','python'), (2, 'Mohammad', 'Java','Docker'), (3, 'Ala','ObjectiveC','Swift')]
devs = spark.createDataFrame(data, ['id', 'name', 'skill_1', 'skill_2'])
devs.show()
devs.printSchema()

+---+--------+----------+-------+
| id|    name|   skill_1|skill_2|
+---+--------+----------+-------+
|  1|   Oveys|      java| python|
|  2|Mohammad|      Java| Docker|
|  3|     Ala|ObjectiveC|  Swift|
+---+--------+----------+-------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skill_1: string (nullable = true)
 |-- skill_2: string (nullable = true)



In [26]:
devs.withColumn('skills', array('skill_1', 'skill_2')).show()

+---+--------+----------+-------+-------------------+
| id|    name|   skill_1|skill_2|             skills|
+---+--------+----------+-------+-------------------+
|  1|   Oveys|      java| python|     [java, python]|
|  2|Mohammad|      Java| Docker|     [Java, Docker]|
|  3|     Ala|ObjectiveC|  Swift|[ObjectiveC, Swift]|
+---+--------+----------+-------+-------------------+



# array_contains()

In [27]:
data = [(1, 'Oveys', ['java', 'python']), (2, 'Mohammad', ['Java', 'Python', 'Docker']), (3, 'Ala',['ObjectiveC', 'Swift', 'swiftUI', 'iOS'])]
devs = spark.createDataFrame(data, ['id', 'name', 'skills'])
devs.show()
devs.printSchema()

+---+--------+--------------------+
| id|    name|              skills|
+---+--------+--------------------+
|  1|   Oveys|      [java, python]|
|  2|Mohammad|[Java, Python, Do...|
|  3|     Ala|[ObjectiveC, Swif...|
+---+--------+--------------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [28]:
def lower_skills(skills):
    return [skill.lower() for skill in skills]


lower_skills_udf = udf(lower_skills, ArrayType(StringType()))

devs.withColumn('skills', lower_skills_udf('skills')).withColumn('pythonist', array_contains('skills', 'python')).show()

+---+--------+--------------------+---------+
| id|    name|              skills|pythonist|
+---+--------+--------------------+---------+
|  1|   Oveys|      [java, python]|     true|
|  2|Mohammad|[java, python, do...|     true|
|  3|     Ala|[objectivec, swif...|    false|
+---+--------+--------------------+---------+



# MapType() and explode() on it:

In [29]:
data = [(1, 'Oveys', {'age':31, 'height':175}), (2, 'Mohammad', {'age':32, 'height':165})]
devs = spark.createDataFrame(data, ['id', 'name', 'properties'])
devs.show(truncate=False)
devs.printSchema()

+---+--------+--------------------------+
|id |name    |properties                |
+---+--------+--------------------------+
|1  |Oveys   |{age -> 31, height -> 175}|
|2  |Mohammad|{age -> 32, height -> 165}|
+---+--------+--------------------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)



In [30]:
devs.select('id', 'name', 'properties', explode('properties')).show(truncate=False)

+---+--------+--------------------------+------+-----+
|id |name    |properties                |key   |value|
+---+--------+--------------------------+------+-----+
|1  |Oveys   |{age -> 31, height -> 175}|age   |31   |
|1  |Oveys   |{age -> 31, height -> 175}|height|175  |
|2  |Mohammad|{age -> 32, height -> 165}|age   |32   |
|2  |Mohammad|{age -> 32, height -> 165}|height|165  |
+---+--------+--------------------------+------+-----+



In [31]:
devs.withColumn('values', map_values('properties')).show(truncate=False)

+---+--------+--------------------------+---------+
|id |name    |properties                |values   |
+---+--------+--------------------------+---------+
|1  |Oveys   |{age -> 31, height -> 175}|[31, 175]|
|2  |Mohammad|{age -> 32, height -> 165}|[32, 165]|
+---+--------+--------------------------+---------+



# when() otherwise()

In [32]:
data = [(1, 'Oveys', 'm'), (2, 'Mohammad', 'm'), (3, 'Ala','f')]
devs = spark.createDataFrame(data, ['id', 'name', 'gender'])
devs.show()
devs.printSchema()

+---+--------+------+
| id|    name|gender|
+---+--------+------+
|  1|   Oveys|     m|
|  2|Mohammad|     m|
|  3|     Ala|     f|
+---+--------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [33]:
devs.select(
    'id',
    'name',
    when(devs.gender=='m', value=True) \
    .when(devs.gender=='f', value=False) \
    .otherwise(value=None).alias('sex')
).show()

+---+--------+-----+
| id|    name|  sex|
+---+--------+-----+
|  1|   Oveys| true|
|  2|Mohammad| true|
|  3|     Ala|false|
+---+--------+-----+



In [34]:
devs.withColumn(
    'gender',
    when(devs.gender=='m', value=True) \
    .when(devs.gender=='f', value=False) \
    .otherwise(value=None).alias('sex')
).show()

+---+--------+------+
| id|    name|gender|
+---+--------+------+
|  1|   Oveys|  true|
|  2|Mohammad|  true|
|  3|     Ala| false|
+---+--------+------+



# column's functions

In [3]:
data = [(1, 'Oveys', 4000), (2, 'Mohammad', 5000), (3, 'Ala',5500)]
devs = spark.createDataFrame(data, ['id', 'name', 'salary'])
devs.show()
devs.printSchema()

+---+--------+------+
| id|    name|salary|
+---+--------+------+
|  1|   Oveys|  4000|
|  2|Mohammad|  5000|
|  3|     Ala|  5500|
+---+--------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [36]:
# alias()
devs.select(devs.id.alias('emp_id'), devs.name, devs.salary).show()

+------+--------+------+
|emp_id|    name|salary|
+------+--------+------+
|     1|   Oveys|  4000|
|     2|Mohammad|  5000|
|     3|     Ala|  5500|
+------+--------+------+



In [37]:
# asc() and desc()
devs.sort(devs.salary.asc()).show()
devs.sort(devs.salary.desc()).show()

+---+--------+------+
| id|    name|salary|
+---+--------+------+
|  1|   Oveys|  4000|
|  2|Mohammad|  5000|
|  3|     Ala|  5500|
+---+--------+------+

+---+--------+------+
| id|    name|salary|
+---+--------+------+
|  3|     Ala|  5500|
|  2|Mohammad|  5000|
|  1|   Oveys|  4000|
+---+--------+------+



In [38]:
# cast()
devs.select('id', 'name', devs.salary.cast('float')).show()

+---+--------+------+
| id|    name|salary|
+---+--------+------+
|  1|   Oveys|4000.0|
|  2|Mohammad|5000.0|
|  3|     Ala|5500.0|
+---+--------+------+



In [6]:
# like()
devs.filter(devs.name.like('A%')).show()

+---+----+------+
| id|name|salary|
+---+----+------+
|  3| Ala|  5500|
+---+----+------+



# filter() and where()

In [40]:
data = [(1, 'Oveys', 4000), (2, 'Mohammad', 5000), (3, 'Ala',5500)]
devs = spark.createDataFrame(data, ['id', 'name', 'salary'])
devs.show()
devs.printSchema()

+---+--------+------+
| id|    name|salary|
+---+--------+------+
|  1|   Oveys|  4000|
|  2|Mohammad|  5000|
|  3|     Ala|  5500|
+---+--------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [41]:
devs.filter('salary == 4000').show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|Oveys|  4000|
+---+-----+------+



In [42]:
devs.filter(devs.salary == 4000).show()


+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|Oveys|  4000|
+---+-----+------+



In [43]:
devs.where((devs.salary >= 5000) & (devs.name.like('A%'))).show()

+---+----+------+
| id|name|salary|
+---+----+------+
|  3| Ala|  5500|
+---+----+------+



# distinct() & drop_duplicates()

In [44]:
data = [(1, 'Oveys','M', 4000), (2, 'Mohammad', 'M', 5000), (2, 'Mohammad', 'M', 5000), (3, 'Ala', 'F', 5500)]
devs = spark.createDataFrame(data, ['id', 'name', 'gender', 'salary'])
devs.show()
devs.printSchema()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|   Oveys|     M|  4000|
|  2|Mohammad|     M|  5000|
|  2|Mohammad|     M|  5000|
|  3|     Ala|     F|  5500|
+---+--------+------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [45]:
devs.distinct().show()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|   Oveys|     M|  4000|
|  2|Mohammad|     M|  5000|
|  3|     Ala|     F|  5500|
+---+--------+------+------+



In [46]:
# drops if there is a duplicate in rows

devs.drop_duplicates().show()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|   Oveys|     M|  4000|
|  2|Mohammad|     M|  5000|
|  3|     Ala|     F|  5500|
+---+--------+------+------+



In [47]:
# drops if there is a duplicate in gender column

devs.drop_duplicates(subset=['gender']).show()

+---+-----+------+------+
| id| name|gender|salary|
+---+-----+------+------+
|  3|  Ala|     F|  5500|
|  1|Oveys|     M|  4000|
+---+-----+------+------+



# union()

In [48]:
data = [(1, 'Oveys','M', 4000), (2, 'Mohammad', 'M', 5000), (2, 'Mohammad', 'M', 5000), (3, 'Ala', 'F', 5500)]
devs1 = spark.createDataFrame(data, ['id', 'name', 'gender', 'salary'])
devs.show()
devs.printSchema()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|   Oveys|     M|  4000|
|  2|Mohammad|     M|  5000|
|  2|Mohammad|     M|  5000|
|  3|     Ala|     F|  5500|
+---+--------+------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [49]:
data = [(1, 'Ali','M', 4000), (2, 'Jafar', 'M', 5000), (2, 'Mohammad', 'M', 5000), (2, 'Abas', 'M', 5000), (3, 'Mona', 'F', 5500)]
devs2 = spark.createDataFrame(data, ['id', 'name', 'gender', 'salary'])
devs2.show()
devs2.printSchema()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|     Ali|     M|  4000|
|  2|   Jafar|     M|  5000|
|  2|Mohammad|     M|  5000|
|  2|    Abas|     M|  5000|
|  3|    Mona|     F|  5500|
+---+--------+------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [50]:
devs1.union(devs2).distinct().show()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|   Oveys|     M|  4000|
|  2|Mohammad|     M|  5000|
|  3|     Ala|     F|  5500|
|  1|     Ali|     M|  4000|
|  2|   Jafar|     M|  5000|
|  2|    Abas|     M|  5000|
|  3|    Mona|     F|  5500|
+---+--------+------+------+



# groupBy() and agg()

In [51]:
data = [
    (1, 'Ali','M', 4000),
    (2, 'Jafar', 'M', 5000),
    (7, 'Mohammad', 'M', 5700),
    (2, 'Abas', 'M', 6500),
    (3, 'Mona', 'F', 6200)]
devs = spark.createDataFrame(data, ['id', 'name', 'gender', 'salary'])
devs.show()
devs.printSchema()

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|     Ali|     M|  4000|
|  2|   Jafar|     M|  5000|
|  7|Mohammad|     M|  5700|
|  2|    Abas|     M|  6500|
|  3|    Mona|     F|  6200|
+---+--------+------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [52]:
devs.groupby(devs.gender).agg(
    count('*').alias('emp_count'),
    min('salary').alias('min_salary'),
    max('salary').alias('max_salary'),
    avg('salary').alias('avg_salary')
).show()

+------+---------+----------+----------+----------+
|gender|emp_count|min_salary|max_salary|avg_salary|
+------+---------+----------+----------+----------+
|     M|        4|      4000|      6500|    5300.0|
|     F|        1|      6200|      6200|    6200.0|
+------+---------+----------+----------+----------+



# unionByName()

In [53]:
data = [(1, 'Ali',20, 4000), (2, 'Jafar', 21, 5000), (2, 'Mohammad', 23, 5000), (2, 'Abas', 45, 5000), (3, 'Mona', 19, 5500)]
devs1 = spark.createDataFrame(data, ['id', 'name', 'age', 'salary'])
devs1.show()
devs1.printSchema()

data = [(1, 'Ali','M', 4000), (2, 'Jafar', 'M', 5000), (2, 'Mohammad', 'M', 5000), (2, 'Abas', 'M', 5000), (3, 'Mona', 'F', 5500)]
devs2 = spark.createDataFrame(data, ['id', 'name', 'gender', 'salary'])
devs2.show()
devs2.printSchema()

+---+--------+---+------+
| id|    name|age|salary|
+---+--------+---+------+
|  1|     Ali| 20|  4000|
|  2|   Jafar| 21|  5000|
|  2|Mohammad| 23|  5000|
|  2|    Abas| 45|  5000|
|  3|    Mona| 19|  5500|
+---+--------+---+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: long (nullable = true)

+---+--------+------+------+
| id|    name|gender|salary|
+---+--------+------+------+
|  1|     Ali|     M|  4000|
|  2|   Jafar|     M|  5000|
|  2|Mohammad|     M|  5000|
|  2|    Abas|     M|  5000|
|  3|    Mona|     F|  5500|
+---+--------+------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [54]:
devs1.unionByName(devs2, allowMissingColumns=True).show()

+---+--------+----+------+------+
| id|    name| age|salary|gender|
+---+--------+----+------+------+
|  1|     Ali|  20|  4000|  NULL|
|  2|   Jafar|  21|  5000|  NULL|
|  2|Mohammad|  23|  5000|  NULL|
|  2|    Abas|  45|  5000|  NULL|
|  3|    Mona|  19|  5500|  NULL|
|  1|     Ali|NULL|  4000|     M|
|  2|   Jafar|NULL|  5000|     M|
|  2|Mohammad|NULL|  5000|     M|
|  2|    Abas|NULL|  5000|     M|
|  3|    Mona|NULL|  5500|     F|
+---+--------+----+------+------+



# self join

In [55]:
data = [(1, 'Ali',0), (2, 'Jafar', 1), (3, 'Mohammad', 1), (4, 'Abas', 3), (5, 'Mona', 2)]
devs = spark.createDataFrame(data, ['id', 'name', 'manager'])
devs.show()
devs.printSchema()

+---+--------+-------+
| id|    name|manager|
+---+--------+-------+
|  1|     Ali|      0|
|  2|   Jafar|      1|
|  3|Mohammad|      1|
|  4|    Abas|      3|
|  5|    Mona|      2|
+---+--------+-------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- manager: long (nullable = true)



In [56]:
devs.alias('emp').join(
    devs.alias('mngr'),
    col('emp.manager') == col('mngr.id'),
    how='left'
).show()

+---+--------+-------+----+--------+-------+
| id|    name|manager|  id|    name|manager|
+---+--------+-------+----+--------+-------+
|  1|     Ali|      0|NULL|    NULL|   NULL|
|  2|   Jafar|      1|   1|     Ali|      0|
|  3|Mohammad|      1|   1|     Ali|      0|
|  4|    Abas|      3|   3|Mohammad|      1|
|  5|    Mona|      2|   2|   Jafar|      1|
+---+--------+-------+----+--------+-------+



# pivot()

In [57]:
data = [(1, 'Ali','M', 'IT'), (2, 'Jafar', 'F', 'HR'), (3, 'Mohammad', 'M', 'HR'), (4, 'Abas', 'F', 'IT'), (5, 'Mona', 'T', 'FIN')]
devs = spark.createDataFrame(data, ['id', 'name', 'gender', 'department'])
devs.show()
devs.printSchema()

+---+--------+------+----------+
| id|    name|gender|department|
+---+--------+------+----------+
|  1|     Ali|     M|        IT|
|  2|   Jafar|     F|        HR|
|  3|Mohammad|     M|        HR|
|  4|    Abas|     F|        IT|
|  5|    Mona|     T|       FIN|
+---+--------+------+----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)



In [58]:
devs.groupBy('department').pivot('gender').count().show()

+----------+----+----+----+
|department|   F|   M|   T|
+----------+----+----+----+
|        HR|   1|   1|NULL|
|       FIN|NULL|NULL|   1|
|        IT|   1|   1|NULL|
+----------+----+----+----+



In [59]:
# we can choose which value of the pivots can be showed as columns
devs.groupBy('department').pivot('gender', ['M', 'F']).count().show()


+----------+----+----+
|department|   M|   F|
+----------+----+----+
|        HR|   1|   1|
|       FIN|NULL|NULL|
|        IT|   1|   1|
+----------+----+----+



# udf()

In [62]:
data = [("1", "03:25:45"), ("2", "01:10:30"), ("3", "05:15:20")]
df = spark.createDataFrame(data, ["id", "duration"])

def duration_to_seconds(duration_str):
    parts = duration_str.split(':')
    hours = int(parts[0])
    minutes = int(parts[1])
    seconds = int(parts[2])
    total_seconds = (hours * 3600) + (minutes * 60) + seconds
    return total_seconds

duration_to_seconds_udf = udf(lambda d: duration_to_seconds(d), returnType=IntegerType())

df.withColumn("duration_seconds", duration_to_seconds_udf("duration")).show()

+---+--------+----------------+
| id|duration|duration_seconds|
+---+--------+----------------+
|  1|03:25:45|           12345|
|  2|01:10:30|            4230|
|  3|05:15:20|           18920|
+---+--------+----------------+



In [63]:
# The reason it works without explicitly registering the UDF is because PySpark automatically 
# converts certain Python functions into UDFs when they are used within DataFrame transformations.
# This behavior is called "automatic UDF registration." 

# Consider example below:
# This demonstrates a scenario where you need to explicitly register the function as a UDF because PySpark cannot automatically
# handle the transformation due to the use of external Python libraries or complex logic. 


In [64]:
import re

data = [("1", "john.doe@example.com"), ("2", "jane.smith@example.com"), ("3", "mike.jones@example.com")]
df = spark.createDataFrame(data, ["id", "email"])

def extract_username(email):
    match = re.match(r'^([a-zA-Z0-9_.+-]+)@', email)
    if match:
        return match.group(1)
    else:
        return None

df_with_username = df.withColumn("username", extract_username(df.email))

df_with_username.show()

TypeError: expected string or bytes-like object, got 'Column'

# using udf for sql temporary tables:

In [65]:
data = [(1, 'Ali',4000, 0), (2, 'Jafar', 2800, 400), (3, 'Mohammad', 3000, 500), (4, 'Abas', 0, 0), (5, 'Mona', 2200, 300)]
devs = spark.createDataFrame(data, ['id', 'name', 'salary', 'bonus'])
devs.show()
devs.printSchema()
devs.createOrReplaceTempView('devs')

+---+--------+------+-----+
| id|    name|salary|bonus|
+---+--------+------+-----+
|  1|     Ali|  4000|    0|
|  2|   Jafar|  2800|  400|
|  3|Mohammad|  3000|  500|
|  4|    Abas|     0|    0|
|  5|    Mona|  2200|  300|
+---+--------+------+-----+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- bonus: long (nullable = true)



In [66]:
def sum_salary(salary: int, bonus: int) -> int:

    return salary + bonus

In [67]:
spark.udf.register(name='udf_sum_salary_sql', f=sum_salary, returnType=IntegerType())

<function __main__.sum_salary(salary: int, bonus: int) -> int>

In [68]:
spark.sql("select id, name,udf_sum_salary_sql(salary, bonus) as sum_salary from devs").show()

+---+--------+----------+
| id|    name|sum_salary|
+---+--------+----------+
|  1|     Ali|      4000|
|  2|   Jafar|      3200|
|  3|Mohammad|      3500|
|  4|    Abas|         0|
|  5|    Mona|      2500|
+---+--------+----------+



# window()

In [69]:
transaction_data = [
    ('2023-01-03', '123123', 227, 'edeka'),
    ('2023-01-03', '123124', 24, 'rewe'),
    ('2023-01-03', '234', 24, 'rewe'),
    ('2023-01-04', '423423', 45, 'edeka'),
    ('2023-01-05', '424332', 65, 'eurogida'),
    ('2023-01-06', '53454', 85, 'aldi'),
    ('2023-01-07', '1534543', 43, 'rewe'),
    ('2023-01-08', '3534434', 32, 'rewe'),
    ('2023-01-08', '56556332', 22, 'rewe'),
    ('2023-01-11', '768679823749', 31, 'rewe'),
    ('2023-01-12', '8652438736478', 54, 'rewe'),
    ('2023-01-13', '93487264823', 41, 'rewe'),
    ('2023-01-09', '122567', 76, 'aldi'),
]

schema = StructType() \
    .add('date', StringType()) \
    .add('t_id', StringType()) \
    .add('price', IntegerType()) \
    .add('shop_name', StringType())

transactions = spark.createDataFrame(
    data=transaction_data,
    schema=schema
)

transactions.show()
transactions.printSchema()

+----------+-------------+-----+---------+
|      date|         t_id|price|shop_name|
+----------+-------------+-----+---------+
|2023-01-03|       123123|  227|    edeka|
|2023-01-03|       123124|   24|     rewe|
|2023-01-03|          234|   24|     rewe|
|2023-01-04|       423423|   45|    edeka|
|2023-01-05|       424332|   65| eurogida|
|2023-01-06|        53454|   85|     aldi|
|2023-01-07|      1534543|   43|     rewe|
|2023-01-08|      3534434|   32|     rewe|
|2023-01-08|     56556332|   22|     rewe|
|2023-01-11| 768679823749|   31|     rewe|
|2023-01-12|8652438736478|   54|     rewe|
|2023-01-13|  93487264823|   41|     rewe|
|2023-01-09|       122567|   76|     aldi|
+----------+-------------+-----+---------+

root
 |-- date: string (nullable = true)
 |-- t_id: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- shop_name: string (nullable = true)



# row_number() rank() dense_rank()

In [70]:
window = Window.orderBy('date', 'price').partitionBy('shop_name')


In [71]:
transactions.select('date', 'shop_name', 'price') \
    .withColumn('first', first('price').over(window)) \
    .withColumn('nth', nth_value('price', 5).over(window)) \
    .withColumn('rank', rank().over(window)) \
    .withColumn('dense', dense_rank().over(window)) \
    .withColumn('percent_rank', percent_rank().over(window)) \
    .withColumn('lead', lead('price', 1).over(window)) \
    .withColumn('lag', lag('price', 1).over(window)) \
    .withColumn('last', last('price').over(window)) \
    .withColumn('row_num', row_number().over(window)).show()

+----------+---------+-----+-----+----+----+-----+-------------------+----+----+----+-------+
|      date|shop_name|price|first| nth|rank|dense|       percent_rank|lead| lag|last|row_num|
+----------+---------+-----+-----+----+----+-----+-------------------+----+----+----+-------+
|2023-01-06|     aldi|   85|   85|NULL|   1|    1|                0.0|  76|NULL|  85|      1|
|2023-01-09|     aldi|   76|   85|NULL|   2|    2|                1.0|NULL|  85|  76|      2|
|2023-01-03|    edeka|  227|  227|NULL|   1|    1|                0.0|  45|NULL| 227|      1|
|2023-01-04|    edeka|   45|  227|NULL|   2|    2|                1.0|NULL| 227|  45|      2|
|2023-01-05| eurogida|   65|   65|NULL|   1|    1|                0.0|NULL|NULL|  65|      1|
|2023-01-03|     rewe|   24|   24|NULL|   1|    1|                0.0|  24|NULL|  24|      1|
|2023-01-03|     rewe|   24|   24|NULL|   1|    1|                0.0|  43|  24|  24|      2|
|2023-01-07|     rewe|   43|   24|NULL|   3|    2| 0.2857142

# apply map() on DF

In [72]:
data = [('Oveys', 'Safarnejad'), ('Ala', 'Nourani')]
schema = StructType().add('first_name', StringType()).add('last_name', StringType())

persons = spark.createDataFrame(data, schema)
persons.show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|     Oveys|Safarnejad|
|       Ala|   Nourani|
+----------+----------+



In [73]:
# dataframes don't support map() on their rows
# You have to convert it to a RDD and then apply map() on them, finally you can reconvert it to dataframe.
persons_maped_rdd = persons.rdd.map(lambda x: (x[0] + ' ' + x[1],))
persons_maped_rdd.toDF(schema=['full_name']).show()

+----------------+
|       full_name|
+----------------+
|Oveys Safarnejad|
|     Ala Nourani|
+----------------+



# partition data on write

In [74]:
data = [(1, 'Ali','M', 'IT'), (2, 'Jafar', 'F', 'HR'), (3, 'Mohammad', 'M', None), (4, 'Abas', 'F', 'IT'), (5, 'Mona', 'T', 'FIN')]
devs = spark.createDataFrame(data, ['id', 'name', 'gender', 'department'])
devs.show()
devs.printSchema()

+---+--------+------+----------+
| id|    name|gender|department|
+---+--------+------+----------+
|  1|     Ali|     M|        IT|
|  2|   Jafar|     F|        HR|
|  3|Mohammad|     M|      NULL|
|  4|    Abas|     F|        IT|
|  5|    Mona|     T|       FIN|
+---+--------+------+----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)



In [75]:
devs.write.parquet('./devs/parquests', partitionBy='department', mode='overwrite')
# This partionBy parameter will devide data into different sections based on department column value.

In [76]:
spark.read.parquet('./devs/parquests').show()

+---+--------+------+----------+
| id|    name|gender|department|
+---+--------+------+----------+
|  3|Mohammad|     M|      NULL|
|  2|   Jafar|     F|        HR|
|  4|    Abas|     F|        IT|
|  5|    Mona|     T|       FIN|
|  1|     Ali|     M|        IT|
+---+--------+------+----------+



In [77]:
spark.read.parquet('./devs/parquests/department=HR').show()

+---+-----+------+
| id| name|gender|
+---+-----+------+
|  2|Jafar|     F|
+---+-----+------+



# from_json()
##### it will accept a column of type string and convert it to MapType or StructType.

In [78]:
data = [
    ('Oveys', '{"age": 31, "hair_color": "black"}'), 
    ('Ala', '{"age": 32, "hair_color": "pink"}'), 
]

persons = spark.createDataFrame(data, schema=['name', 'props'])

persons.printSchema()
persons.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- props: string (nullable = true)

+-----+----------------------------------+
|name |props                             |
+-----+----------------------------------+
|Oveys|{"age": 31, "hair_color": "black"}|
|Ala  |{"age": 32, "hair_color": "pink"} |
+-----+----------------------------------+



### The question is to convert values in the map type column into different columns ?

In [79]:


keys = persons.select(
    explode(
        map_keys(
            from_json(
                'props',
                schema=MapType(StringType(),StringType())
            )
        )
    )
).distinct().rdd.flatMap(lambda x: x).collect()
exprs = [from_json(col('props'),schema=MapType(StringType(),StringType())).getItem(k).alias(k) for k in keys]
persons.select('name', *exprs).show()




+-----+----------+---+
| name|hair_color|age|
+-----+----------+---+
|Oveys|     black| 31|
|  Ala|      pink| 32|
+-----+----------+---+



In [80]:
# this way is more optimized since it is not calling collect() which may cause performance issue on large datasets.

json_schema = spark.read.json(persons.rdd.map(lambda r: r.props)).schema
persons = persons.withColumn("_c", from_json("props", json_schema))
persons.select("name", "_c.*").show()

+-----+---+----------+
| name|age|hair_color|
+-----+---+----------+
|Oveys| 31|     black|
|  Ala| 32|      pink|
+-----+---+----------+



# to_json()
###### It will convert a column from MapType or StructType to json string

In [81]:
data = [
    ('Oveys', {"age": 31, "hair_color": "black"}), 
    ('Ala', {"age": 32, "hair_color": "pink"}), 
]

persons = spark.createDataFrame(data, schema=['name', 'props'])

persons.printSchema()
persons.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- props: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)

+-----+-------------------------------+
|name |props                          |
+-----+-------------------------------+
|Oveys|{age -> 31, hair_color -> NULL}|
|Ala  |{age -> 32, hair_color -> NULL}|
+-----+-------------------------------+



In [82]:
persons.withColumn('json_props', to_json('props')).show(truncate=0)

+-----+-------------------------------+----------------------------+
|name |props                          |json_props                  |
+-----+-------------------------------+----------------------------+
|Oveys|{age -> 31, hair_color -> NULL}|{"age":31,"hair_color":null}|
|Ala  |{age -> 32, hair_color -> NULL}|{"age":32,"hair_color":null}|
+-----+-------------------------------+----------------------------+



# json_tuple()

In [83]:
data = [
    ('Oveys', '{"personal_info": {"hair_color":"black", "eye_color":"black"}, "age":31}'),
    ('Ala', '{"personal_info":{"hair_color":"brown", "eye_color":"brown"}, "age":32}')
]
schema = StructType(
    [StructField('name', StringType()), StructField('props', StringType())]
)

persons = spark.createDataFrame(data, schema)

persons.printSchema()
persons.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- props: string (nullable = true)

+-----+------------------------------------------------------------------------+
|name |props                                                                   |
+-----+------------------------------------------------------------------------+
|Oveys|{"personal_info": {"hair_color":"black", "eye_color":"black"}, "age":31}|
|Ala  |{"personal_info":{"hair_color":"brown", "eye_color":"brown"}, "age":32} |
+-----+------------------------------------------------------------------------+



In [84]:
persons.select('name', json_tuple('props', 'age').alias('age')).show()

+-----+---+
| name|age|
+-----+---+
|Oveys| 31|
|  Ala| 32|
+-----+---+



# get_json_object()

In [85]:
persons.select('name', sql_functions.get_json_object('props', '$.personal_info.hair_color').alias('hair')).show()

+-----+-----+
| name| hair|
+-----+-----+
|Oveys|black|
|  Ala|brown|
+-----+-----+



#### but how to extract all the keys as columns?

## method 1 (manual - based on keys) 

In [86]:
json_schema = spark.read.json(persons.rdd.map(lambda r: r.props)).schema
persons.withColumn("_c", from_json("props", json_schema)).select("name", "_c.age", "_c.personal_info.*").show()


+-----+---+---------+----------+
| name|age|eye_color|hair_color|
+-----+---+---------+----------+
|Oveys| 31|    black|     black|
|  Ala| 32|    brown|     brown|
+-----+---+---------+----------+



## method 2 (Dynamic key extracting)

In [138]:
def get_schema_all_keys(base_schema: StructType, keys: list=[], parent_key="") -> list:
    for key in base_schema.fields:
        if isinstance(key.dataType, StructType):
            new_parent_key = f"{parent_key}.{key.name}" if parent_key else key.name
            get_schema_all_keys(key.dataType, keys, new_parent_key)
        else:
            full_key = f"{parent_key}.{key.name}" if parent_key else key.name
            keys.append(full_key)
    return keys

In [140]:
data = [
    ('Oveys', '{"personal_info": {"hair_color":"black", "eye_color":"black", "size":80}, "age":31}'),
    ('Ala', '{"personal_info":{"hair_color":"brown", "eye_color":"brown"}, "age":32}')
]

persons = spark.createDataFrame(data, schema=['name', 'props'])

props_schema = spark.read.json(persons.rdd.map(lambda r: r.props)).schema
keys = get_schema_all_keys(props_schema)
for key in keys:
    col_name = key.split('.')[-1]
    persons = persons.withColumn(col_name, sql_functions.expr(f"get_json_object(props, '$.{key}')"))

persons.show()

+-----+--------------------+---+---------+----------+----+
| name|               props|age|eye_color|hair_color|size|
+-----+--------------------+---+---------+----------+----+
|Oveys|{"personal_info":...| 31|    black|     black|  80|
|  Ala|{"personal_info":...| 32|    brown|     brown|NULL|
+-----+--------------------+---+---------+----------+----+



# date functions
##### default date format is yyyy-MM-dd

In [3]:
data = [(1, 'Ali','M', 'IT'), (2, 'Jafar', 'F', 'HR'), (3, 'Mohammad', 'M', None), (4, 'Abas', 'F', 'IT'), (5, 'Mona', 'T', 'FIN')]
devs = spark.createDataFrame(data, ['id', 'name', 'gender', 'department'])
devs.show()
devs.printSchema()

+---+--------+------+----------+
| id|    name|gender|department|
+---+--------+------+----------+
|  1|     Ali|     M|        IT|
|  2|   Jafar|     F|        HR|
|  3|Mohammad|     M|      NULL|
|  4|    Abas|     F|        IT|
|  5|    Mona|     T|       FIN|
+---+--------+------+----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)



In [4]:
devs = devs.withColumn('today', sql_functions.current_date())
devs.show()

+---+--------+------+----------+----------+
| id|    name|gender|department|     today|
+---+--------+------+----------+----------+
|  1|     Ali|     M|        IT|2024-04-05|
|  2|   Jafar|     F|        HR|2024-04-05|
|  3|Mohammad|     M|      NULL|2024-04-05|
|  4|    Abas|     F|        IT|2024-04-05|
|  5|    Mona|     T|       FIN|2024-04-05|
+---+--------+------+----------+----------+



In [6]:
devs.withColumn('Unformated', sql_functions.date_format(col('today'), 'yyyy')).show()

+---+--------+------+----------+----------+----------+
| id|    name|gender|department|     today|Unformated|
+---+--------+------+----------+----------+----------+
|  1|     Ali|     M|        IT|2024-04-05|      2024|
|  2|   Jafar|     F|        HR|2024-04-05|      2024|
|  3|Mohammad|     M|      NULL|2024-04-05|      2024|
|  4|    Abas|     F|        IT|2024-04-05|      2024|
|  5|    Mona|     T|       FIN|2024-04-05|      2024|
+---+--------+------+----------+----------+----------+



In [14]:
devs = devs.withColumn('lit_date', sql_functions.to_date(lit('05.07.1992'), 'dd.MM.yyyy'))
devs.show()

+---+--------+------+----------+----------+----------+
| id|    name|gender|department|     today|  lit_date|
+---+--------+------+----------+----------+----------+
|  1|     Ali|     M|        IT|2024-04-05|1992-07-05|
|  2|   Jafar|     F|        HR|2024-04-05|1992-07-05|
|  3|Mohammad|     M|      NULL|2024-04-05|1992-07-05|
|  4|    Abas|     F|        IT|2024-04-05|1992-07-05|
|  5|    Mona|     T|       FIN|2024-04-05|1992-07-05|
+---+--------+------+----------+----------+----------+



In [15]:
devs.withColumn('age', sql_functions.date_diff('today', 'lit_date')).show()

+---+--------+------+----------+----------+----------+-----+
| id|    name|gender|department|     today|  lit_date|  age|
+---+--------+------+----------+----------+----------+-----+
|  1|     Ali|     M|        IT|2024-04-05|1992-07-05|11597|
|  2|   Jafar|     F|        HR|2024-04-05|1992-07-05|11597|
|  3|Mohammad|     M|      NULL|2024-04-05|1992-07-05|11597|
|  4|    Abas|     F|        IT|2024-04-05|1992-07-05|11597|
|  5|    Mona|     T|       FIN|2024-04-05|1992-07-05|11597|
+---+--------+------+----------+----------+----------+-----+



In [16]:
devs.withColumn('month_between', sql_functions.months_between('today', 'lit_date')).show()

+---+--------+------+----------+----------+----------+-------------+
| id|    name|gender|department|     today|  lit_date|month_between|
+---+--------+------+----------+----------+----------+-------------+
|  1|     Ali|     M|        IT|2024-04-05|1992-07-05|        381.0|
|  2|   Jafar|     F|        HR|2024-04-05|1992-07-05|        381.0|
|  3|Mohammad|     M|      NULL|2024-04-05|1992-07-05|        381.0|
|  4|    Abas|     F|        IT|2024-04-05|1992-07-05|        381.0|
|  5|    Mona|     T|       FIN|2024-04-05|1992-07-05|        381.0|
+---+--------+------+----------+----------+----------+-------------+



In [13]:
devs.withColumn('add_month', sql_functions.add_months('today', 3)).show()

+---+--------+------+----------+----------+----------+----------+
| id|    name|gender|department|     today|  lit_date| add_month|
+---+--------+------+----------+----------+----------+----------+
|  1|     Ali|     M|        IT|2024-04-05|1993-08-18|2024-07-05|
|  2|   Jafar|     F|        HR|2024-04-05|1993-08-18|2024-07-05|
|  3|Mohammad|     M|      NULL|2024-04-05|1993-08-18|2024-07-05|
|  4|    Abas|     F|        IT|2024-04-05|1993-08-18|2024-07-05|
|  5|    Mona|     T|       FIN|2024-04-05|1993-08-18|2024-07-05|
+---+--------+------+----------+----------+----------+----------+



In [17]:
devs.withColumn('sub_month', sql_functions.add_months('today', -2)).show()

+---+--------+------+----------+----------+----------+----------+
| id|    name|gender|department|     today|  lit_date| sub_month|
+---+--------+------+----------+----------+----------+----------+
|  1|     Ali|     M|        IT|2024-04-05|1992-07-05|2024-02-05|
|  2|   Jafar|     F|        HR|2024-04-05|1992-07-05|2024-02-05|
|  3|Mohammad|     M|      NULL|2024-04-05|1992-07-05|2024-02-05|
|  4|    Abas|     F|        IT|2024-04-05|1992-07-05|2024-02-05|
|  5|    Mona|     T|       FIN|2024-04-05|1992-07-05|2024-02-05|
+---+--------+------+----------+----------+----------+----------+



In [18]:
devs.withColumn('days_add', sql_functions.date_add('today', 10)).show()


+---+--------+------+----------+----------+----------+----------+
| id|    name|gender|department|     today|  lit_date|  days_add|
+---+--------+------+----------+----------+----------+----------+
|  1|     Ali|     M|        IT|2024-04-05|1992-07-05|2024-04-15|
|  2|   Jafar|     F|        HR|2024-04-05|1992-07-05|2024-04-15|
|  3|Mohammad|     M|      NULL|2024-04-05|1992-07-05|2024-04-15|
|  4|    Abas|     F|        IT|2024-04-05|1992-07-05|2024-04-15|
|  5|    Mona|     T|       FIN|2024-04-05|1992-07-05|2024-04-15|
+---+--------+------+----------+----------+----------+----------+



#### dayOfYear(), dayOfWeek(), dayOfYear(), days(), month(), year() are other functions ...

## default TimestampType format is yyyy-MM-dd HH:mm:ss.SS

In [4]:
df = spark.range(4)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
+---+



In [5]:
df = df.withColumn('timestamp', sql_functions.current_timestamp())
df.show(truncate=False)

+---+--------------------------+
|id |timestamp                 |
+---+--------------------------+
|0  |2024-04-05 15:48:39.067556|
|1  |2024-04-05 15:48:39.067556|
|2  |2024-04-05 15:48:39.067556|
|3  |2024-04-05 15:48:39.067556|
+---+--------------------------+



In [6]:
df = df.withColumn('from_lit', sql_functions.to_timestamp(lit('08-18-1993 12:00:04.04'), format='MM-dd-yyyy HH:mm:ss.SS'))
df.show(truncate=False)
df.printSchema()

+---+--------------------------+----------------------+
|id |timestamp                 |from_lit              |
+---+--------------------------+----------------------+
|0  |2024-04-05 15:48:39.460769|1993-08-18 12:00:04.04|
|1  |2024-04-05 15:48:39.460769|1993-08-18 12:00:04.04|
|2  |2024-04-05 15:48:39.460769|1993-08-18 12:00:04.04|
|3  |2024-04-05 15:48:39.460769|1993-08-18 12:00:04.04|
+---+--------------------------+----------------------+

root
 |-- id: long (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- from_lit: timestamp (nullable = true)



In [7]:
df.select(
    '*',
    sql_functions.hour('from_lit').alias('hour'),
    sql_functions.minute('from_lit').alias('minutes'),
    sql_functions.second('from_lit').alias('seconds'),
).show(truncate=False)

+---+--------------------------+----------------------+----+-------+-------+
|id |timestamp                 |from_lit              |hour|minutes|seconds|
+---+--------------------------+----------------------+----+-------+-------+
|0  |2024-04-05 15:48:40.526679|1993-08-18 12:00:04.04|12  |0      |4      |
|1  |2024-04-05 15:48:40.526679|1993-08-18 12:00:04.04|12  |0      |4      |
|2  |2024-04-05 15:48:40.526679|1993-08-18 12:00:04.04|12  |0      |4      |
|3  |2024-04-05 15:48:40.526679|1993-08-18 12:00:04.04|12  |0      |4      |
+---+--------------------------+----------------------+----+-------+-------+



# aggregations

In [10]:
data = [(1, 'Ali','M', 'IT', 1200), (2, 'Jafar', 'F', 'HR', 20000), (3, 'Mohammad', 'M', None, 30000), (4, 'Abas', 'F', 'IT',  200), (5, 'Mona', 'T', 'FIN', 6000)]
devs = spark.createDataFrame(data, ['id', 'name', 'gender', 'department', 'salary'])
devs.show()
devs.printSchema()

+---+--------+------+----------+------+
| id|    name|gender|department|salary|
+---+--------+------+----------+------+
|  1|     Ali|     M|        IT|  1200|
|  2|   Jafar|     F|        HR| 20000|
|  3|Mohammad|     M|      NULL| 30000|
|  4|    Abas|     F|        IT|   200|
|  5|    Mona|     T|       FIN|  6000|
+---+--------+------+----------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [20]:
devs.select(sql_functions.approx_count_distinct('department')).show()
devs.select(sql_functions.approx_count_distinct('gender')).show()

+---------------------------------+
|approx_count_distinct(department)|
+---------------------------------+
|                                3|
+---------------------------------+

+-----------------------------+
|approx_count_distinct(gender)|
+-----------------------------+
|                            3|
+-----------------------------+



In [13]:
devs.select(sql_functions.avg('salary')).show()

+-----------+
|avg(salary)|
+-----------+
|    11480.0|
+-----------+



In [22]:
devs.select(sql_functions.collect_list('salary')).show(truncate=False)

+-------------------------------+
|collect_list(salary)           |
+-------------------------------+
|[1200, 20000, 30000, 200, 6000]|
+-------------------------------+



In [17]:
devs.select(sql_functions.collect_set('salary')).show(truncate=False)

+-------------------------------+
|collect_set(salary)            |
+-------------------------------+
|[1200, 30000, 6000, 20000, 200]|
+-------------------------------+



In [18]:
devs.select(sql_functions.count_distinct('department')).show(truncate=False)

+--------------------------+
|count(DISTINCT department)|
+--------------------------+
|3                         |
+--------------------------+



In [19]:
devs.select(sql_functions.count('department')).show(truncate=False)


+-----------------+
|count(department)|
+-----------------+
|4                |
+-----------------+



# Errors and Missing Values

In [35]:
from pyspark.sql.functions import isnan, when, count, col

integrated_df = spark.read.option('header', True).csv("./bom_sources/source_1.csv")
integrated_df.show()
# # Handling Missing Values
# # Count missing values in each column
missing_counts_for_each_col = integrated_df.select([count(when(isnan(c) |  col(c).isNull(), 1)).alias(c) for c in integrated_df.columns])
missing_counts_for_each_col.show(200)


# # Removing Duplicate Entries
# # Drop duplicate rows based on selected columns
cleaned_df = integrated_df.dropDuplicates(['Part_Number'])


# # Validating Quantity Values
# # Check for out-of-range quantity values
valid_quantity_df = cleaned_df.filter((col("Quantity") >= 20) & (col("Quantity") <= 40))
valid_quantity_df.show(50)

# # Verifying Part Numbers
# # Validate part numbers based on predefined patterns
valid_part_numbers_df = valid_quantity_df.filter(col("Part_Number").rlike("[0-9]{3}"))
valid_part_numbers_df.show(50)

# # Cross-Referencing with Reference Data (Example: Supplier Information)
# # Join with reference data to validate supplier information
# reference_supplier_df = spark.read.parquet("supplier_data.parquet")
# validated_bom_df = valid_part_numbers_df.join(reference_supplier_df, "Component_ID", "left_outer")

# # Checking Hierarchical Relationships (Example: Assembly Quantity Check)
# # Validate assembly quantities against sum of sub-component quantities
# assembly_quantity_check_df = validated_bom_df.withColumn("Quantity_Check", when(
#     col("Component_Type") == "Assembly", col("Quantity") == col("Subcomponent_Quantities").sum()).otherwise(True))

# # Write the cleaned and validated data to a new Parquet file
# assembly_quantity_check_df.write.parquet("cleaned_validated_bom.parquet")

# # Stop the Spark session
# spark.stop()


+-----------+------------+--------------------+--------+-------+----------+------------+-----+
|Part_Number|        Name|         Description|Quantity|    EOL|Unit_Price|Manufacturer|avail|
+-----------+------------+--------------------+--------+-------+----------+------------+-----+
|   12312312|   'sdadasd'|'sadas asdsad asdas'|      12|01-2026|       123| 'company 1'|    1|
|   45645645|    'qwerty'|'Lorem ipsum dolo...|      30|06-2025|        89| 'company 2'|    1|
|   78978978|    'foobar'|'Consectetur adip...|      25|09-2027|        55| 'company 3'|    0|
|   10101010|       'xyz'|'Sed do eiusmod t...|      18|04-2024|        72| 'company 4'|    1|
|   11111111|    'abcdef'|'Ut enim ad minim...|      40|12-2026|       105| 'company 5'|    1|
|   12121212|    'ghijkl'|'Quis nostrud exe...|      20|03-2025|        95| 'company 6'|    1|
|   13131313|    'mnopqr'|'Duis aute irure ...|      15|08-2023|        80| 'company 7'|    1|
|   14141414|    'stuvwx'|'Excepteur sint o...|   

In [ ]:
# titanic_1.fillna(value='GG', subset='cabin').show()